<a href="https://colab.research.google.com/github/dercodeKoenig/colab-KI/blob/main/c%C3%A4ser_vs_KI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verschlüsseln

In [ ]:
text="Kuenstliche Neuronen besiegen Caeser-Verschluesselung".lower()
schlüssel = 4

verschlüsselt = ""
a = list(text.encode('UTF-8'))
for i in a:
  if(chr(i).isalpha()):
    tmp=chr(i+schlüssel)
    if(tmp.isalpha()==False):
      tmp=chr(i+schlüssel-26)
    verschlüsselt+=tmp
  else:
    verschlüsselt+=chr(i)

print(verschlüsselt)

oyirwxpmgli riyvsrir fiwmikir geiwiv-zivwglpyiwwipyrk


Entschlüsseln<br>


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

leise = True #kurze Ausgabe

#Modell laden
#Code zum trainieren des Modells auf https://colab.research.google.com/github/dercodeKoenig/colab-KI/blob/main/text_ki.ipynb
#die Parameter werden so importiert wie sie im trainingsnotebook gespeichert wurden
!wget -q https://github.com/dercodeKoenig/colab-KI/raw/main/weights.hdf5

#Diese Werte sind im Modell eintrainiert
seq_length = 5
raw_text2="QWERTZUIOPÜ*'ASDFGHJKLÖÄYXCVBNM;:_qwertzuiopü+#asdfghjklöäyxcvbnm,.-<>|@1234567890ß!\"§$%&/()=? \n»«~+*\\—½{}[]è" #alle zeichen hier rein

chars = sorted(list(set(raw_text2)))
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_vocab = len(chars)
print ("Anzahl unterstützter Buchstaben: ", n_vocab)



filepath="weights.hdf5"

#Modell-Struktur für die vortrainierten Parameter
#Dropout, loss und Optimierer können rausgelassen werden, weil sie nur im Training wichtig sind

model = Sequential()
model.add(LSTM(32, input_shape=(seq_length, n_vocab),return_sequences=True))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(2, activation='softmax'))
model.compile()
try:
  model.load_weights(filepath)
except:
    print("fehler")

#print(model.summary())


max_score=-1
bester_schlüssel=0
bester_text=""




print("")
print("Entschlüsselung starten")
print("")
verschlüsselt_kopie = verschlüsselt # Eingabe des Verschlüsselten Texts, wird aus oberer Code-Zeile übernommen

for a in range(26): #alles durchtesten
  verschlüsselt= verschlüsselt_kopie
  schlüssel = a

  entschlüsselt = ""

  a = list(verschlüsselt.encode('UTF-8'))

  for i in a:
    if(chr(i).isalpha()):
      tmp=chr(i+schlüssel)
      if(tmp.isalpha()==False):
        tmp=chr(i+schlüssel-26)
      entschlüsselt+=tmp
    else:
      entschlüsselt+=chr(i)



  eingabe = entschlüsselt
  if(leise==False):
    print(eingabe)
  n_chars = len(eingabe)
  dataX = [] #liste aus Sequenzen für die KI
  for i in range(n_chars-seq_length):
    data=eingabe[i:i+seq_length];
    dataX+=data
  dataX=np.reshape(np.array(dataX),(int(len(dataX)/seq_length),seq_length))
  Xeingabe = []
  for i in range(len(dataX)):
    for o in range(len(dataX[i])):
      Xeingabe+=[char_to_int[dataX[i][o]]]
  Xeingabe = to_categorical(Xeingabe,num_classes=n_vocab)
  #  X = [samples, time steps, features]
  Xeingabe = np.reshape(Xeingabe, (int(len(Xeingabe)/seq_length), seq_length, n_vocab)) #DataX one-hot-encoded -> aus 2 wird 3 Dimensionen

  wahr = 0.0
  falsch = 0.0
  gesamt = 0

  for i in Xeingabe:
    gesamt+=1
    Xeingabe=np.reshape(np.array(i),(1,seq_length,n_vocab)) #Die KI erwartet ein "batch" aus Eingabedatenweil sie durchaus mit TPUs mehrere Eingaben gleichzeitig bearbeitet
                                                            #Die Eingabe muss nach X = [samples, time steps, features] aufgebaut sein, nur dass nur 1 "sample" durchgerechnet wird
                                                            #alternativ zu dieser Schleife könnte man auch alle Auf einmal durchführen aber das ist jetzt so nicht eingebaut
                                                            #wäre was für die Zukunft
    res = model.predict(Xeingabe)

    wahr += res[0][1]                                       # 2 Neuronen in res sagen falsch und wahr mit einer wahrscheinlichkeit
                                                            # Wenn falsch von wahr abgezogen wird hat man ein Ergebnis zwischen -1 und 1 für falsch und richtig
    falsch +=res[0][0]
  if((wahr - falsch)/gesamt>max_score):
    max_score = (wahr - falsch)/gesamt
    bester_schlüssel = schlüssel
    bester_text = entschlüsselt
  
  if(leise==False):
    print("Bewertung Text: ",((wahr - falsch)/gesamt))
    print("")
  else:
    print("\r", schlüssel, " / 25: ",(wahr - falsch)/gesamt,", bester Schlüssel: ", bester_schlüssel, " ("+bester_text[0:15]+"...)", end="")
  
if(leise):
  print("\rfertig!")
print("")
print("")
print("Ergebnis:")
print(bester_text)
print("")
print("score: ", max_score)
print("Schlüssel: ", bester_schlüssel)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '§', '«', '»', '½', 'Ä', 'Ö', 'Ü', 'ß', 'ä', 'è', 'ö', 'ü', '—']
Anzahl unterstützter Buchstaben:  107

Entschlüsselung starten

fertig!


Ergebnis:
kuenstliche neuronen besiegen caeser-verschluesselung

score:  0.9951673729586202
Schlüssel:  22
